In [1]:
import numpy as np
import pandas as pd

path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT10] Series de Tiempo\Clases prácticas\Práctica 3-20250511\apm.txt"
df = pd.read_csv(path, delimiter="\t", decimal=".")
df.head()

,Name,cons,r1,r10,r2,r3,r4,r5,r6,r7,r8,r9,rf
0,NaN,1.00203,0.032754,0.004037,0.017294,0.031547,0.025391,0.038211,0.034982,0.024625,0.035583,0.017122,0.002223
1,NaN,1.01293,0.016348,0.002989,0.021230,0.023306,0.020960,0.007620,0.010083,0.012738,0.003211,0.007321,0.002304
2,NaN,0.99169,0.026965,0.044662,0.014533,0.010366,0.034431,0.028731,0.034575,0.036312,0.042527,0.018228,0.002426
3,NaN,1.00867,0.001786,0.027943,0.020739,-0.005343,0.004528,0.014246,-0.000680,0.017128,0.009693,-0.003238,0.002336
4,NaN,0.99797,-0.010474,-0.003220,0.004005,0.005863,0.008635,0.007676,0.014163,0.009478,0.004531,0.014151,0.002636


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    0 non-null      float64
 1   cons    418 non-null    float64
 2   r1      418 non-null    float64
 3   r10     418 non-null    float64
 4   r2      418 non-null    float64
 5   r3      418 non-null    float64
 6   r4      418 non-null    float64
 7   r5      418 non-null    float64
 8   r6      418 non-null    float64
 9   r7      418 non-null    float64
 10  r8      418 non-null    float64
 11  r9      418 non-null    float64
 12  rf      418 non-null    float64
dtypes: float64(13)
memory usage: 42.6 KB


In [38]:
from scipy.optimize import differential_evolution
from scipy.stats import norm, chi2
import statsmodels.stats.sandwich_covariance as smcov
from statsmodels.stats.stattools import durbin_watson

# --- 0) Datos ---
df = df.copy()
T = len(df)
cons = df['cons'].values
rf   = df['rf'].values
R    = np.column_stack([df[f"r{i}"] for i in range(1, 11)])  # (T,10)

# --- 1) Función de momentos ---
def mean_mom(theta):
    c1, c2 = theta
    base = c1 * cons ** (-c2)
    m1 = base * (1 + rf) - 1
    m_rest = base[:, None] * (R - rf[:, None])
    return np.concatenate([[m1.mean()], m_rest.mean(axis=0)])  # (11,)

# --- 2) Función GMM con identidad ---
def Q(theta):
    g = mean_mom(theta)
    return np.dot(g, g)

# --- 3) Estimación de parámetros ---
bounds = [(0.1, 5.0), (0.1, 200.0)]
res_de = differential_evolution(Q, bounds, maxiter=1000, tol=1e-12, seed=42)
c1_hat, c2_hat = res_de.x

# --- 4) Momentos evaluados en el óptimo ---
g11 = mean_mom([c1_hat, c2_hat])
J_stat = T * np.dot(g11, g11)
p_J = 1 - chi2.cdf(J_stat, df=9)

# --- 5) Matriz de momentos para HAC ---
w = c1_hat * cons ** (-c2_hat)
M11 = np.column_stack([
    w * (1 + rf) - 1,
    *(w * (R[:, i] - rf) for i in range(10))
])
S = smcov.S_hac_simple(M11, nlags=0, weights_func=smcov.weights_bartlett)

# --- 6) Jacobiana numérica centrada ---
eps = 1e-6
D = np.zeros((11, 2))
theta0 = np.array([c1_hat, c2_hat])
for j in range(2):
    step = np.zeros(2)
    step[j] = eps
    D[:, j] = (mean_mom(theta0 + step) - mean_mom(theta0 - step)) / (2 * eps)

# --- 7) Varianza GMM (sandwich robusto) ---
A = D.T @ D
try:
    A_inv = np.linalg.inv(A)
except np.linalg.LinAlgError:
    A_inv = np.linalg.pinv(A)  # regulariza si es singular

B = D.T @ S @ D
V = np.linalg.inv(A) @ B @ np.linalg.inv(A)  # quitar el / T

# --- 8) Errores estándar, t y p ---
se_hac   = np.sqrt(np.diag(V))
t_stats  = np.array([c1_hat, c2_hat]) / se_hac
p_values = 2 * (1 - norm.cdf(np.abs(t_stats)))

# --- 9) Determinante de la matriz de varianzas ---
Sigma_hat = (M11.T @ M11) / T
det_Sigma = np.linalg.det(Sigma_hat)

# --- 10) Reporte Final ---
print("    Coefficient    Std. Error    t-Statistic    Prob.")
print(f"C(1)    {c1_hat: .6f}    {se_hac[0]: .5f}    {t_stats[0]: .5f}    {p_values[0]: .4f}")
print(f"C(2)    {c2_hat: .6f}    {se_hac[1]: .2f}    {t_stats[1]: .5f}    {p_values[1]: .4f}\n")
print(f"Determinant residual covariance    {det_Sigma: .2E}")
print(f"J-statistic                        {J_stat: .6f}")
print(f"p-value (Chi2_9)                   {p_J: .4f}\n")

# --- 11) Diagnóstico por ecuación ---
for i in range(11):
    resid = M11[:, i]
    ssr = np.sum(resid**2)
    se_reg = np.sqrt(ssr / (T - 2))
    dw = durbin_watson(resid)
    label = "1+RF" if i == 0 else f"R{i}-RF"
    print(f"Equation {i+1:2d}: {label}")
    print(f"  S.E. of regression    {se_reg:.6f}    Sum squared resid    {ssr:.4f}")
    print(f"  Durbin-Watson stat    {dw:.6f}\n")

    Coefficient    Std. Error    t-Statistic    Prob.
C(1)     0.699606     60.01072     0.01166     0.9907
C(2)     91.409728     15933.60     0.00574     0.9954

Determinant residual covariance     1.54E-36
J-statistic                         0.001866
p-value (Chi2_9)                    1.0000

Equation  1: 1+RF
  S.E. of regression    0.666577    Sum squared resid    184.8392
  Durbin-Watson stat    1.687835

Equation  2: R1-RF
  S.E. of regression    0.094381    Sum squared resid    3.7056
  Durbin-Watson stat    1.914519

Equation  3: R2-RF
  S.E. of regression    0.084154    Sum squared resid    2.9460
  Durbin-Watson stat    1.844234

Equation  4: R3-RF
  S.E. of regression    0.078703    Sum squared resid    2.5767
  Durbin-Watson stat    1.855824

Equation  5: R4-RF
  S.E. of regression    0.075941    Sum squared resid    2.3991
  Durbin-Watson stat    1.782310

Equation  6: R5-RF
  S.E. of regression    0.072116    Sum squared resid    2.1635
  Durbin-Watson stat    1.795585
